# Selecting Optimal Upgrades:
## *Data Processing and Risk Assignment*

In [2]:
###########
## Setup ##
###########

import arcpy
import os
#aprx = arcpy.mp.ArcGISProject(r"C:\ArcGIS_Pro\Conda\Cloned_Envs\Selecting_Optimal_Upgrades_RTS_GMLC\Selecting_Optimal_Upgrades_RTS_GMLC.aprx")
#m = aprx.listMaps("Map")[0]

In [4]:
#################################
## Add RTSGMLC branches to map ##
#################################

# in_json_file = r"C:\Users\sofie\OneDrive\Documents\UW\Research\RTS GMLC Files\RTS-GMLC-master\RTS-GMLC-master\RTS_Data\FormattedData\GIS\branch.geojson"
# out_features = "RTSGMLC_branch"

# arcpy.conversion.JSONToFeatures(in_json_file, out_features, "POLYLINE")

# Be sure to delete transformers here

In [2]:
# Split lines into 10km segments
arcpy.GeneratePointsAlongLines_management("RTSGMLC_branch", "RTSGMLC_branch_10km_pts.shp", 'DISTANCE', Distance='10 kilometers')
arcpy.SplitLineAtPoint_management("RTSGMLC_branch", "RTSGMLC_branch_10km_pts.shp", "RTSGMLC_branch_10km.shp", "0.1 Meters")

<Result 'C:\\ArcGIS_Pro\\Conda\\Cloned_Envs\\Selecting_Optimal_Upgrades_RTS_GMLC\\RTSGMLC_branch_10km.shp'>

In [2]:
# Split lines into 10km segments
arcpy.GeneratePointsAlongLines_management("TL_8_16_21", "GeoGrid_CA_branch_10km_pts.shp", 'DISTANCE', Distance='10 kilometers')
arcpy.SplitLineAtPoint_management("TL_8_16_21", "GeoGrid_CA_branch_10km_pts.shp", "GeoGrid_CA_branch_10km.shp", "0.1 Meters")

<Result 'C:\\ArcGIS_Pro\\Conda\\Cloned_Envs\\Selecting_Optimal_Upgrades_RTS_GMLC\\GeoGrid_CA_branch_10km.shp'>

In [3]:
# Split lines into 1km segments
arcpy.GeneratePointsAlongLines_management("RTSGMLC_branch", "RTSGMLC_branch_1km_pts.shp", 'DISTANCE', Distance='1 kilometers')
arcpy.SplitLineAtPoint_management("RTSGMLC_branch", "RTSGMLC_branch_1km_pts.shp", "RTSGMLC_branch_1km.shp", "0.1 Meters")

<Result 'C:\\ArcGIS_Pro\\Conda\\Cloned_Envs\\Selecting_Optimal_Upgrades_RTS_GMLC\\RTSGMLC_branch_1km.shp'>

In [3]:
# Split lines into 1km segments
arcpy.GeneratePointsAlongLines_management("TL_8_16_21", "GeoGrid_CA_branch_1km_pts.shp", 'DISTANCE', Distance='1 kilometers')
arcpy.SplitLineAtPoint_management("TL_8_16_21", "GeoGrid_CA_branch_1km_pts.shp", "GeoGrid_CA_branch_1km.shp", "0.1 Meters")

<Result 'C:\\ArcGIS_Pro\\Conda\\Cloned_Envs\\Selecting_Optimal_Upgrades_RTS_GMLC\\GeoGrid_CA_branch_1km.shp'>

In [2]:
#######################
## Process WFPI Data ##
#######################

# Specify start and end dates for WFPI maps
start_date = datetime.date(2021, 1, 1)
end_date = datetime.date(2021, 1, 2) # end date is one day after you want to end
delta = datetime.timedelta(days=1)

date = start_date

# Start program timer
start_time = time.time()

while date < end_date:
    num = date.strftime("%Y%m%d")
    
    # Skip these missing dates
    if (num != '20210602') and (num != '20210607') and (num != '20211007') and (num < '20210601'):

        ##########################
        ## Add WFPI data to map ##
        ##########################

        path = rf'C:\Users\lucas\Documents\2021_Wind-enhanced_Fire_Potential_Index_Forecast_1_DATA\emodis_wfpi_data_{num}_{num}.tiff'
        arcpy.MakeRasterLayer_management(path, f"wfpi_{num}_tiff")
        arcpy.Clip_management(f"wfpi_{num}_tiff", "-2051218 -2120000 20000 40000", f"wfpi_{num}_clipped_tiff", "#", "#", "NONE", "NO_MAINTAIN_EXTENT")

        #########################################
        ## Convert WFPI from Raster to Polygon ##
        #########################################

        arcpy.RasterToPolygon_conversion(f"wfpi_{num}_tiff", f"wfpi_{num}.shp", "NO_SIMPLIFY", "VALUE")

        #################################################
        ## Modify gridcodes that indicate no fire risk ##
        #################################################
        # The WFPI data gridcodes of 248 and above indicate unburnable land. 
        # We will change them to be negative values.

        cursor = arcpy.UpdateCursor(f"wfpi_{num}.shp")
        for row in cursor: 
            if row.getValue("gridcode") >= 248:
                row.setValue("gridcode", 0)
                cursor.updateRow(row)
                
    if (num != '20210602') and (num != '20210607') and (num != '20211007') and (num != '20211010') and (num != '20211014') and (num > '20210621'):
        
        ##########################
        ## Add WFPI data to map ##
        ##########################

        path = rf'C:\Users\lucas\Documents\2021_Wind-enhanced_Fire_Potential_Index_Forecast_1_DATA\emodis-wfpi_data_{num}_{num}.tiff'
        arcpy.MakeRasterLayer_management(path, f"wfpi_{num}_tiff")
        arcpy.Clip_management(f"wfpi_{num}_tiff", "-2051218 -2120000 20000 40000", f"wfpi_{num}_clipped_tiff", "#", "#", "NONE", "NO_MAINTAIN_EXTENT")

        #########################################
        ## Convert WFPI from Raster to Polygon ##
        #########################################

        arcpy.RasterToPolygon_conversion(f"wfpi_{num}_tiff", f"wfpi_{num}.shp", "NO_SIMPLIFY", "VALUE")

        #################################################
        ## Modify gridcodes that indicate no fire risk ##
        #################################################
        # The WFPI data gridcodes of 248 and above indicate unburnable land. 
        # We will change them to be negative values.

        cursor = arcpy.UpdateCursor(f"wfpi_{num}.shp")
        for row in cursor: 
            if row.getValue("gridcode") >= 248:
                row.setValue("gridcode", 0)
                cursor.updateRow(row)
                
    # Go to next day
    #if (num == '20210602') or (num == '20210607') or (num == '20211007') or (num == '20211010') or (num == '20211014') and (num >= '20210601'):
    date += delta
    
print("--- %s seconds ---" % (time.time() - start_time))

--- 225.93692469596863 seconds ---


In [6]:
#####################################
## Assign Risk to RTSGMLC branches ##
#####################################

import os

# Specify start and end dates for WFPI maps
start_date = datetime.date(2021, 10, 15)
end_date = datetime.date(2021, 11, 7)
delta = datetime.timedelta(days=1)

date = start_date

# Start program timer
start_time = time.time()

while date < end_date:
    num = date.strftime("%Y%m%d")
    
    # Skip these missing dates
    if (num != '20210602') and (num != '20210607') and (num != '20211007') and (num != '20211010') and (num != '20211014'):

    #####################################################
    ## Assign max WFPI risk value to RTS-GMLC branches ##
    #####################################################

        pace = r"C:\Users\lucas\Documents\ArcGIS\Projects\LFranke_WISPO_Proj"

        if date == start_date:
            target_features = "branch_JSONToFeatures"
        else:
            prev_date = date - delta
            prevnum = prev_date.strftime("%Y%m%d")
            target_features = f"RTSGMLC_Max_1km_{prevnum}"

        join_features = os.path.join(pace, f"wfpi_{num}.shp")
        out_fc = f"RTSGMLC_Max_1km_{num}"

        # Field mappings specify how attributes from both classes will be combined in the output
        fieldmappings = arcpy.FieldMappings()
        fieldmappings.addTable(target_features)
        fieldmappings.addTable(join_features)

        gridcodeFieldIndex = fieldmappings.findFieldMapIndex("gridcode")
        fieldmap = fieldmappings.getFieldMap(gridcodeFieldIndex)

        field = fieldmap.outputField

        field.name = f"max_WFPI_{num}"
        field.aliasName = f"max_WFPI_{num}"
        fieldmap.outputField = field

        fieldmap.mergeRule = "Max"
        fieldmappings.replaceFieldMap(gridcodeFieldIndex, fieldmap)

        # Delete fields that are no longer applicable
        x = fieldmappings.findFieldMapIndex("Id")
        fieldmappings.removeFieldMap(x)

        arcpy.analysis.SpatialJoin(target_features, join_features, out_fc, "#", "#", fieldmappings)
        # Go to next day
    date += delta
    if (num == '20210602') or (num == '20210607') or (num == '20211007') or (num == '20211010') or (num == '20211014') and (num > '20210601'):
        # Go to next day
        date += delta
    
print("--- %s seconds ---" % (time.time() - start_time))

--- 198.16512870788574 seconds ---


In [8]:
### Convert risk into analyzable tables ###

import os
import arcpy

# Specify start and end dates for WFPI maps
start_date = datetime.date(2021, 1, 1)
end_date = datetime.date(2021, 1, 2)
delta = datetime.timedelta(days=1)

date = start_date

# Start program timer
start_time = time.time()

while date < end_date:
    num = date.strftime("%Y%m%d")
    
    # Skip these missing dates
    if (num != '20210602') and (num != '20210607') and (num != '20211007'):

        ##########################
        ## Convert Risk to Table and Export ##
        ##########################
        arcpy.env.workplace =  r"C:\Users\lucas\Documents\ArcGIS\Projects\LFranke_WISPO_Proj"
        arcpy.TableToTable_conversion(f"RTSGMLCs_Max_1km_{num}", "C:\Users\lucas\Documents\ArcGIS\Projects\LFranke_WISPO_Proj\Default.gdb", f"Risk_Table_{num}")

                
    # Go to next day
    date += delta
    
print("--- %s seconds ---" % (time.time() - start_time))

SyntaxError: (unicode error) 'unicodeescape' codec can't decode bytes in position 2-3: truncated \UXXXXXXXX escape (<string>, line 26)

In [2]:
# Specify start and end dates for WFPI maps
start_date = datetime.date(2021, 8, 21)
end_date = datetime.date(2021, 9, 1)
delta = datetime.timedelta(days=1)

date = start_date

# Start program timer
start_time = time.time()

while date < end_date:
# for num in range(startnum, endnum+1):
    num = date.strftime("%Y%m%d")

    #######################################################
    ## Assign max WFPI risk value to GeoGrid-CA branches ##
    #######################################################

    workspace = r"C:\ArcGIS_Pro\Conda\Cloned_Envs\Selecting_Optimal_Upgrades_RTS_GMLC"

    #if date == start_date:
    #    target_features = "GeoGridCA_1km"
    #else:
    prev_date = date - delta
    prevnum = prev_date.strftime("%Y%m%d")
    target_features = f"GeoGridCA_Max_1km_{prevnum}"

    join_features = os.path.join(workspace, f"wfpi_{num}.shp")
    out_fc = f"GeoGridCA_Max_1km_{num}"

    # Field mappings specify how attributes from both classes will be combined in the output
    fieldmappings = arcpy.FieldMappings()
    fieldmappings.addTable(target_features)
    fieldmappings.addTable(join_features)

    gridcodeFieldIndex = fieldmappings.findFieldMapIndex("gridcode")
    fieldmap = fieldmappings.getFieldMap(gridcodeFieldIndex)

    field = fieldmap.outputField

    field.name = f"max_WFPI_{num}"
    field.aliasName = f"max_WFPI_{num}"
    fieldmap.outputField = field

    fieldmap.mergeRule = "Max"
    fieldmappings.replaceFieldMap(gridcodeFieldIndex, fieldmap)

    # Delete fields that are no longer applicable
    x = fieldmappings.findFieldMapIndex("Id")
    fieldmappings.removeFieldMap(x)

    arcpy.analysis.SpatialJoin(target_features, join_features, out_fc, "#", "#", fieldmappings)

    # Go to next day
    date += delta
    
print("--- %s seconds ---" % (time.time() - start_time))

ExecuteError: ERROR 999999: Something unexpected caused the tool to fail. Contact Esri Technical Support (http://esriurl.com/support) to Report a Bug, and refer to the error help for potential solutions or workarounds.
Failed to execute (SpatialJoin).


In [6]:
table = 
"RTSGMLC_Cm_NoSgmt_20210701".gridcode[1]

AttributeError: 'str' object has no attribute 'gridcode'